In [89]:
#Bayesian classifier

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import math

from skimage import transform
from scipy.stats import multivariate_normal
from scipy.stats import norm
from mpmath import mp

def unpickle(file):
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding="latin1")
    return dict

#datadict = unpickle('/home/kamarain/Data/cifar-10-batches-py/data_batch_1')
datadict = unpickle('C:/Users/danie/Desktop/TUNI/ML-PatternRec/EX2/cifar-10-batches-py/test_batch')

X_test = datadict["data"]
Y_test = datadict["labels"]

#labeldict = unpickle('/home/kamarain/Data/cifar-10-batches-py/batches.meta')
labeldict = unpickle('C:/Users/danie/Desktop/TUNI/ML-PatternRec/EX2/cifar-10-batches-py/batches.meta')
label_names = labeldict["label_names"]


#X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
#X = X.reshape(len(X), 3, 32, 32).transpose(0, 2, 3, 1).astype("uint8") moved to cifar_10_color



input_directory= ('C:/Users/danie/Desktop/TUNI/ML-PatternRec/EX2/cifar-10-batches-py/')
train_1= unpickle(input_directory+'data_batch_1')
train_2= unpickle(input_directory+'data_batch_2')
train_3= unpickle(input_directory+'data_batch_3')
train_4= unpickle(input_directory+'data_batch_4')
train_5= unpickle(input_directory+'data_batch_5')

#train_X=np.append(train_1["data"],train_2["data"],train_3["data"],train_4["data"],train_5["data"],axis=0)
train_X=np.append(train_1["data"],train_2["data"],axis=0)
train_X=np.append(train_X, train_3["data"],axis=0)
train_X=np.append(train_X, train_4["data"],axis=0)
train_X=np.append(train_X, train_5["data"],axis=0)


#print(type(train_1["labels"]))
train_Y=train_1["labels"]
train_Y.extend(train_2["labels"])
train_Y.extend(train_3["labels"])
train_Y.extend(train_4["labels"])
train_Y.extend(train_5["labels"])

train_Y = np.array(train_Y)
Y = np.array(Y)
print(len(train_Y))


50000


In [90]:
def cifar_10_color(X,n=1):
    color_output=[]
    X1=X.copy()
    X1 = X1.reshape(X1.shape[0], 3, 32, 32).transpose(0,2,3,1).astype(np.uint8)
    for i in range(X1.shape[0]):
        resi_resha=transform.resize(X1[i],(n,n)).reshape(n*n*3)
        color_output.append(resi_resha)
    return np.array(color_output)
    

In [91]:
def cifar_10_naivebayes_learn(X,Y,isNaive=True):
    mean=[]
    var=[]
    priori=[]
    cov=[]
    data=np.c_[X,Y]
    #np.c_ arrays will be stacked along their last axis after being upgraded to at least 2-D
    for i in range(len(label_names)):
        filter =np.array([i])
        mask = np.in1d(data[:, data.shape[1]-1], filter)
        data_1=data[mask]
        data_2=[]
        current_mean=[]
        
        for j in range(data_1.shape[1]-1):
            current_mean.append(data_1[:,j].mean())
           
            if not isNaive:
                data_2.append(data_1[:,j])
        mean.append(current_mean)
        priori.append(data_1.shape[0]/Y.shape[0])
        current_var=[]
        data_2=np.array(data_2)
      
        for j in range(data_1.shape[1]-1):
            if isNaive:
                current_var.append(data_1[:,j].var())
        if isNaive:
            var.append(current_var)
        else:
            var.append(np.cov(data_2,bias=True))

    return np.array(mean), np.array(var), np.array(priori)

In [92]:
def gaussian_PDF(x,mean,var):
      return (1/math.sqrt(2*math.pi*var))*math.exp(-0.5*((x-mean)**2/var))

In [93]:
def multivariate_normal_dist(x, d, mean, cov):
    x_m = x - mean
    return (1. / (mp.sqrt((2 * np.pi)**d * np.linalg.det(cov))) * mp.exp(-(np.linalg.solve(cov, x_m).T.dot(x_m)) / 2))
    # np.linalg.det - compute determinant of array

In [94]:
def cifar10_classifier_naivebayes(X,mean, var, apriori, isNaive=True):
    Y=[]
    for i in range(len(X)):
        pred_rgb=[]
        for j in range(len(label_names)):
            if isNaive:
                temp_loop_pred=1
                for k in range(mean.shape[1]):
                    loop_pred=gaussian_PDF(X[i,k],mean[j,k],var[j,k])
                    temp_loop_pred=temp_loop_pred*loop_pred
                pred_rgb.append(temp_loop_pred*apriori[j])
            else:
                
                current_var = multivariate_normal(mean=mean[j], cov=var[j]).logpdf(X[i])
                #Log of the probability density function evaluated at x
                
                pred_rgb.append(current_var*apriori[j])
            
        sum_rgb=sum(pred_rgb)
        max_pred=-1
        max_label=None
        for j in range(len(label_names)):
            current_pred=pred_rgb[j]/sum_rgb
            if current_pred>max_pred:
                max_pred=current_pred
                max_label=j
        Y.append(max_label)
        
    return np.array(Y)

In [95]:
def class_acc(prediction,ground_truth):
    temp=prediction-ground_truth
    temp1=temp[temp==0]
    return (temp1.size/prediction.size)*100

In [96]:
rgb_mean_training=cifar_10_color(train_X,n=1)
mean,var,apriori=cifar_10_naivebayes_learn(rgb_mean_training,train_Y)

rgb_mean_testing=cifar_10_color(X_test,n=1)
prediction=cifar10_classifier_naivebayes(rgb_mean_testing, mean, var, apriori)

acc=class_acc(prediction,Y_test)
print("naive bayes - accuracy: ==> "+str(acc))

naive bayes - accuracy: ==> 19.54


In [97]:
rgb_mean_training=cifar_10_color(train_X)
mean,var,apriori=cifar_10_naivebayes_learn(rgb_mean_training,train_Y,isNaive=False)

rgb_mean_testing=cifar_10_color(X_test)
prediction=cifar10_classifier_naivebayes(rgb_mean_testing,mean,var,apriori,isNaive=False)

acc=class_acc(prediction,Y_test)
print("bayes multivariate - accuracy: ==> "+str(acc))

bayes multivariate - accuracy: ==> 20.95


In [ ]:
N=list(range(2,16))
N_1=[1]
acc_list=[acc]
for i in N:
    try:
        rgb_mean_training=cifar_10_color(train_X,n=i)
        mean,var,apriori=cifar_10_naivebayes_learn(rgb_mean_training, train_Y, isNaive=False)
        
        rgb_mean_testing=cifar_10_color(X_test,n=i)
        prediction=cifar10_classifier_naivebayes(rgb_mean_testing,mean,var,apriori,isNaive=False)
        
        acc=class_acc(prediction, Y_test)
        print(" bayes multivariate with n==> "+str(i)+" accuracy: ==> "+str(acc))
        acc_list.append(acc)
        N_1.append(i)
    except:
        print("exception occured when N = "+str(i)+" skipping the rest")
        break

 bayes multivariate with n==> 2 accuracy: ==> 30.11
 bayes multivariate with n==> 3 accuracy: ==> 38.65
 bayes multivariate with n==> 4 accuracy: ==> 39.0
 bayes multivariate with n==> 5 accuracy: ==> 39.77
 bayes multivariate with n==> 6 accuracy: ==> 40.47
 bayes multivariate with n==> 7 accuracy: ==> 40.489999999999995
 bayes multivariate with n==> 8 accuracy: ==> 40.75
 bayes multivariate with n==> 9 accuracy: ==> 41.06
 bayes multivariate with n==> 10 accuracy: ==> 41.48
 bayes multivariate with n==> 11 accuracy: ==> 41.44
 bayes multivariate with n==> 12 accuracy: ==> 41.6


In [ ]:
plt.plot(np.array(N_1),np.array(acc_list))